In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

data = h5py.File('/home/max/Mount/VMC_with_LPTFs/output_8x8/Ising/64-B=512-K=2562x2/3/samples.h5', 'r')
print(len(data.keys()))

In [ ]:
every = 1
for step in range(len(data.keys()))[::every]:
    # Average over the batch dimension and reshape to square lattice
    samples_avg = np.mean(np.array(data[f"step_{step:05d}"]), axis=0).reshape(8,8, order="C")
    
    # Plot the average lattice
    plt.imshow(samples_avg, cmap='gray')
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"Lattice Step {step}")
    plt.grid()
    plt.show()